In [1]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
## set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'Exp2Pilot3'


## Sanity Checks

In [ ]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'},
                                {'iterationName':'Exp2Pilot1'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

### Find full datasets for Silhouette_1

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName': iterationName}]
                     })
complete_data_df = pd.DataFrame(query)
#complete_data_ids = list(complete_data_df['workerId'])

In [ ]:
complete_data_df['workerId'].nunique()

In [ ]:
complete_data_df

## Collect data from db and filter with sanity checks

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative','buildTime',
                                                            'currBonus','exploreResets','buildResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'buildStartTime','buildFinishTime','targetName','numBlocksExplore']]

df = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])


### Integrate reset data before sending

In [ ]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'build'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with build data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetBuildBlocks'})

pre_reset_blocks = pre_reset_blocks.fillna(value={'preResetBuildBlocks': 0})
df = df.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')

df2.merge(df1, on['a'], how='left')

# Rename and add totals
df = df.fillna(value={'preResetBuildBlocks': 0})
df = df.rename(columns = {'numBlocks':'finalBuildBlocks'})
df['totalBuildBlocks'] = df['finalBuildBlocks'] + df['preResetBuildBlocks'].apply(np.sum)

In [ ]:
df

In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df_for_analysis.to_csv(out_path)

## Settled Block Data

In [ ]:
print('Loading iteration: ' + iterationName)
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'settled'},
                        {'iterationName':iterationName}]
                     })

df_settled_full = pd.DataFrame(list(query))


# filter dataframe for complete datasets
df_settled_full_filtered = df_settled_full[df_settled_full.workerId.isin(complete_data_ids)]

print('Loaded ' + str(df_settled_full_filtered.shape[0]) + ' complete sets of settled blocks')
# reduce to crucial information

In [ ]:

df_settled_reduced_filtered = df_settled_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','numBlocks', 'timeAbsolute','timeRelative',
                                                            'normedScore','currBonus','score','incrementalScore','normedIncrementalScore',
                                                            'currBonus','allVertices','targetName','relativePlacementTime','iterationName',
                                                            'blockKind'
                                                           ]]

df_settled_reduced_filtered = df_settled_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

buildstart = df_for_analysis[['gameID','trialNum','buildStartTime','exploreStartTime']]

# copy across time variables that were not saved in the correct place in pilot 3
df_settled_reduced_filtered = df_settled_reduced_filtered.merge(buildstart, on=['gameID', 'trialNum'], how='left')
df_settled_reduced_filtered['timePlaced'] = df_settled_reduced_filtered['timeAbsolute'] - df_settled_reduced_filtered['buildStartTime']
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced < 0),'timePlaced'] = df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['timeAbsolute'] - df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['exploreStartTime']

# 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.iterationName == 'pilot4'),'timePlaced'] = df_settled_reduced_filtered[(df_settled_reduced_filtered.iterationName == 'pilot4')]['relativePlacementTime'] 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced <= 0),'timePlaced'] = 0



In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df_settled_reduced_filtered.to_csv(out_path)

In [ ]:
## load in dataframe
data_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df = pd.read_csv(data_path)
df

## Explore End

In [ ]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'}]},
                        {'trialNum':0}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))

assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [ ]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName': iterationName},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName':iterationName}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_explore_end_full_filtered = df_explore_end_full[df_explore_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_explore_end_reduced_filtered = df_explore_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative',
                                                            'currBonus','exploreResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'targetName','numBlocksExplore']]

df_explore = df_explore_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

In [ ]:
df_explore

## Integrate reset data before sending

In [ ]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'explore'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with explore data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetExploreBlocks'})
df_explore = df_explore.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')

# Rename and add totals
df_explore.loc[df_explore.preResetExploreBlocks.isnull(),'preResetExploreBlocks'] = df_explore.preResetExploreBlocks.loc[df_explore.preResetExploreBlocks.isnull()].apply(lambda x:[])
df_explore = df_explore.rename(columns = {'numBlocksExplore':'finalExploreBlocks'})
df_explore['totalExploreBlocks'] = df_explore['finalExploreBlocks'] + df_explore['preResetExploreBlocks'].apply(np.sum)

#df_explore = df_explore.fillna(value={'totalExploreBlocks': 0 })


In [ ]:
df_explore['numAttempts'] = df_explore['preResetExploreBlocks'].apply(len) + 1

In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_explore_{}.csv'.format(iterationName))
df_explore.to_csv(out_path)

### Test data saving

In [ ]:
iterationName = 'Exp2Pilot2_turk'

In [ ]:
query = coll.find({"$and":[
                        #{'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })
df = pd.DataFrame(list(query.sort('timeAbsolute')))
df

In [ ]:
query = coll.find({"$and":[
                        #{'condition':{'$ne':'practice'}},
                        {'eventType':'initial'},
                        {'iterationName':iterationName}]
                     })
df = pd.DataFrame(list(query.sort('timeAbsolute')))
df.columns

In [ ]:
df[['gameID','trialNum','phase','condition',
    'eventType','score','normedScore','numBlocks',
    'timeAbsolute','timeRelative','buildTime',
    'currBonus','allVertices','nPracticeAttempts',
    'buildStartTime','buildFinishTime','targetName',
    'blockColor','blockColorID','numTargets', 'prePostSetSize',
    'numRepetitions','repetition','targetID',
    'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
    'timeThresholdYellow','timeThresholdRed','devMode',
    'timeBonus'
   ]]

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df = pd.DataFrame(list(query.sort('gameID')))

# df[['gameID','trialNum','phase','condition',
#     'eventType','score','normedScore','numBlocks',
#     'timeAbsolute','timeRelative','buildTime',
#     'currBonus','allVertices','nPracticeAttempts',
#     'buildStartTime','buildFinishTime','targetName',
#     'blockColor','blockColorID','numTargets', 'prePostSetSize',
#     'numRepetitions','repetition','targetID',
#     'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
#     'timeThresholdYellow','timeThresholdRed','devMode',
#     'timeBonus'
#    ]]

df[['gameID','trialNum','phase','condition','repetition','doNothingRepeats','normedScore','normedScoreDiscrete','targetName','currBonus','timeBonus','score','browser','browserVersion','os']]



In [5]:
# get sequence of trial_end data for all people
# note that these currently lump everyone together, and don't
query = coll.find({"$and":[
                        {'eventType':'survey_data'},
                        {'iterationName':iterationName}]
                     })
df_survey = pd.DataFrame(list(query.sort('absoluteTime')))
df_survey[['gameID','age','comments','difficulty','fun','strategies','inputDevice','sex','score']]

,gameID,age,comments,difficulty,fun,strategies,inputDevice,sex,score
0,1691-3673775e-bf17-46c9-b775-79afc1355b02,28,"Hard! Much harder than I expected! I loved it,...",6,7,Unfortunately I didn't.,Mouse,Male,0.325
1,6834-b18d5535-d036-4aa4-bd46-e31404e1a9b0,50,It was vry diificult nut fun I like to use ny ...,6,7,A good base and trying to pay attention to det...,Trackpad,Female,0.090
2,9415-40177ff4-0b4d-494c-96b4-3409ff007cdb,28,I enjoyed the hit.,4,6,I just saw which piece will be suitable for th...,Mouse,Female,0.070
3,6679-27a62967-b9d8-4a25-8b63-74e16e2f7f32,30,"It was fine, no issues",2,5,redrawing the pattern by imagining that I was ...,Mouse,Female,0.635
4,0823-6fe08630-db1c-47e9-b895-7a2bf343bb77,43,"Frustrating at first, but got the hang of it a...",5,6,Not really. Started from the bottom and would...,Mouse,Male,0.805
5,4739-25f27c31-0d4c-46ae-a515-02351c69042d,25,It was horrible. Unless you got it 100% accura...,7,1,I tried my best at first and realized that unl...,Mouse,Male,0.060
6,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,24,,7,5,,Mouse,Female,0.050
7,1106-c12ab2b8-d38b-4e8f-9244-5adfbadd1093,31,,5,6,just eyeballed it,Mouse,Female,0.405
8,3988-e15c8e2e-0b53-43fd-a2d3-686d3efd6923,50,I thought it was fun no problems with it,6,6,Just tried to follow picture the best I could,Mouse,Female,0.000
9,9995-aa7dd1d6-ff3f-41e8-ade6-da416e7366c3,30,,4,5,no,Mouse,Female,0.170


In [ ]:
list(df_survey[['workerId','age','comments','difficulty','fun','strategies','inputDevice','sex','score']]['comments'])

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'workerId':"A2UCTX06NM6Y02"},
                        {'iterationName':iterationName}]
                     })

df = pd.DataFrame(list(query.sort('gameID')))

# df[['gameID','trialNum','phase','condition',
#     'eventType','score','normedScore','numBlocks',
#     'timeAbsolute','timeRelative','buildTime',
#     'currBonus','allVertices','nPracticeAttempts',
#     'buildStartTime','buildFinishTime','targetName',
#     'blockColor','blockColorID','numTargets', 'prePostSetSize',
#     'numRepetitions','repetition','targetID',
#     'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
#     'timeThresholdYellow','timeThresholdRed','devMode',
#     'timeBonus'
#    ]]

#df[['gameID','trialNum','phase','condition','repetition','doNothingRepeats','normedScore','normedScoreDiscrete','targetName','currBonus','timeBonus','score','browser','browserVersion','os']]

df['score']
